In [1]:
print("""
@File         : exercise_12_finding_outliers.ipynb
@Author(s)    : Stephen CUI
@LastEditor(s): Stephen CUI
@CreatedTime  : 2024-10-03 23:03:11
@Email        : cuixuanstephen@gmail.com
@Description  : 
""")


@File         : exercise_12_finding_outliers.ipynb
@Author(s)    : Stephen CUI
@LastEditor(s): Stephen CUI
@CreatedTime  : 2024-10-03 23:03:11
@Email        : cuixuanstephen@gmail.com
@Description  : 



In [2]:
import pandas as pd
import numpy as np

In [3]:
trip_distance = pd.read_csv("../DATA/taxi-distance.csv", header=None).squeeze()
passenger_count = pd.read_csv("../DATA/taxi-passenger-count.csv", header=None).squeeze()

In [4]:
df = pd.DataFrame({"trip_distance": trip_distance, "passenger_count": passenger_count})
df

,trip_distance,passenger_count
0,1.63,1
1,0.46,1
2,0.87,1
3,2.13,1
4,1.40,1
...,...,...
9994,2.70,1
9995,4.50,1
9996,5.59,1
9997,1.54,6


In [6]:
iqr = (df.trip_distance.quantile(.75) - df.trip_distance.quantile(.25))
iqr

2.3

In [7]:
df.loc[
    df.trip_distance < df.trip_distance.quantile(.25) - 1.5 * iqr
]

,trip_distance,passenger_count


In [8]:
df.loc[
    df.trip_distance > df.trip_distance.quantile(.75) + 1.5 * iqr
]

,trip_distance,passenger_count
7,11.90,4
60,9.30,1
73,12.65,1
82,10.24,3
88,23.76,2
...,...,...
9975,7.60,1
9976,12.60,1
9979,11.30,1
9980,9.13,1


In [9]:
df.loc[
    df.trip_distance > df.trip_distance.quantile(.75) + 1.5 * iqr,
    'passenger_count'
]

7       4
60      1
73      1
82      3
88      2
       ..
9975    1
9976    1
9979    1
9980    1
9982    1
Name: passenger_count, Length: 1219, dtype: int64

In [10]:
df.loc[
    df.trip_distance > df.trip_distance.quantile(.75) + 1.5 * iqr,
    'passenger_count'
].mean()

1.7301066447908122

Beyond

In [11]:
df[(df.trip_distance < df.trip_distance.quantile(.1)) |
   (df.trip_distance > df.trip_distance.quantile(.9))]

,trip_distance,passenger_count
1,0.46,1
7,11.90,4
9,0.60,1
10,0.01,3
13,0.50,2
...,...,...
9976,12.60,1
9978,0.38,1
9979,11.30,1
9980,9.13,1


If you’re only interested in removing the non-outlier values, you can use the scipy.stats.trimboth function on your series. It takes a second argument: the proportion you want to cut from both the top and bottom.

In [14]:
from scipy.stats import trimboth

trimboth(df.trip_distance, .1)

(8001,)

In [15]:
from scipy.stats import zscore
df.trip_distance.loc[abs(zscore(df.trip_distance)) > 3]

88      23.76
238     18.32
379     16.38
509     16.82
641     19.72
        ...  
9897    16.11
9899    17.48
9906    17.70
9955    15.49
9964    18.55
Name: trip_distance, Length: 306, dtype: float64

## NaN and missing data

我们如何在 Pandas 中表示缺失值？使用 0 很诱人，但你可以想象，当我们尝试计算平均值时，这会带来麻烦。

相反，pandas 使用一种称为 `NaN` 的东西，也就是不是数字。你可以说 `np.nan` 或 `np.NaN`；pandas 传统上更喜欢后者。无论你怎么写，它仍然是 `np.nan`。**这个奇怪的值是一个浮点数，不能转换成整数，也不等于它自己。**

> pandas 核心开发人员建议他们将来会从 `NaN` 切换到他们自己的 `pd.NA` 值，这是使用比NumPy 更灵活的内部 pandas 数据类型的一部分。

在 NumPy 中，我们通常使用 `isnan` 函数搜索 `NaN` 值。不过，Pandas 采用了不同的方法：我们可以使用 `fillna` 方法替换序列（或数据框）中的 `NaN` 值，并且可以使用 `dropna` 方法删除任何包含 `NaN` 值的行。

这些方法返回一个新的 Series 或数据框，而不是修改原始对象。但是，我们返回的新对象可能没有复制数据，这意味着赋值给它可能会引发著名的、可怕的 `SettingWithCopyWarning`。如果打算修改从 `df.dropna` 返回的 Series 或数据框，可能应该调用 `copy` 方法，以确保安全：

```python
df = df.dropna().copy()
```

可以想象，删除任何包含单个 `NaN` 值的行都可能非常极端。因此，`dropna` 方法有一个 `thresh` 参数，我们可以向其传
递一个整数：一行必须包含的良好非 `NaN` 值的数量，才能保留该行。可能需要认真考虑要如何严格地过滤数据。

> Series 上的 `count` 方法返回非 `NaN` 值的数量。如果没有 `NaN` 值，则结果与系列的大小相同。数据框上的 `count` 方法返回以列名称作为索引的系列。如果任何列的 `count` 结果低于其他列，那是因为它们包含 `NaN` 值。